In [9]:
# import urllib.request
# urllib.request.urlretrieve('https://raw.githubusercontent.com/bab2min/corpus/master/sentiment/naver_shopping.txt', 'shopping.txt')

In [17]:
import pandas as pd
import numpy as np
import tensorflow as tf
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

raw = pd.read_table('shopping.txt', names=['rating', 'review'])
print(raw)

raw['label'] = np.where(raw['rating'] > 3, [1], 0) # 조건에 따라 새로운 열을 만들어준다. 3점 이상이면 1, 아니면 0
print(raw)

Default GPU Device: /device:GPU:0
        rating                                             review
0            5                                            배공빠르고 굿
1            2                      택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2            5  아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...
3            2  선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...
4            5                  민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ
...        ...                                                ...
199995       2                                    장마라그런가!!! 달지않아요
199996       5  다이슨 케이스 구매했어요 다이슨 슈퍼소닉 드라이기 케이스 구매했어요가격 괜찮고 배송...
199997       5                    로드샾에서 사는것보다 세배 저렴하네요 ㅜㅜ 자주이용할께요
199998       5                                      넘이쁘고 쎄련되보이네요~
199999       5   아직 사용해보지도않았고 다른 제품을 써본적이없어서 잘 모르겠지만 ㅎㅎ 배송은 빨랐습니다

[200000 rows x 2 columns]
        rating                                             review  label
0            5                                            배공빠르고 굿      1
1

In [19]:
# 데이터 전처리
raw['review'] = raw['review'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣0-9 ]', '') # 1. 한글과 공백을 제외하고 모두 제거

# raw.isnull().sum() # 2. 결측치 확인

raw.drop_duplicates( subset=['review'], inplace=True ) # 3. 중복값 제거
print(raw)

#bag of words # 4. 유니크한 단어들을 추출
유니크문자 = raw['review'].tolist()
유니크문자 = ''.join(유니크문자)
유니크문자 = list(set(유니크문자))
유니크문자.sort()
print(유니크문자[0:100])

        rating                                             review  label
0            5                                            배공빠르고 굿      1
1            2                      택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고      0
2            5  아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...      1
3            2  선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...      0
4            5                  민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ      1
...        ...                                                ...    ...
199995       2                                    장마라그런가!!! 달지않아요      0
199996       5  다이슨 케이스 구매했어요 다이슨 슈퍼소닉 드라이기 케이스 구매했어요가격 괜찮고 배송...      1
199997       5                    로드샾에서 사는것보다 세배 저렴하네요 ㅜㅜ 자주이용할께요      1
199998       5                                      넘이쁘고 쎄련되보이네요~      1
199999       5   아직 사용해보지도않았고 다른 제품을 써본적이없어서 잘 모르겠지만 ㅎㅎ 배송은 빨랐습니다      1

[199908 rows x 3 columns]
['\x02', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1'

In [20]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(char_level=True, oov_token='<OOV>') # True = 글자단위를 정수로 변환, False = 단어단위를 정수로 변환

문자리스트 = raw['review'].tolist() # 문자를 리스트로 변환
tokenizer.fit_on_texts(문자리스트) # 리스트로 변환된 문자를 정수로 변환

print(tokenizer.word_index) # 각 글자에 대한 인덱스를 부여
print(문자리스트[0:10])

train_seq = tokenizer.texts_to_sequences(문자리스트[0:10]) # 각 글자에 대한 인덱스를 부여
print(train_seq[0:10])

Y = raw['label'].tolist()
print(Y[0:10])

{'<OOV>': 1, ' ': 2, '요': 3, '.': 4, '이': 5, '고': 6, '다': 7, '아': 8, '는': 9, '어': 10, '하': 11, '니': 12, '가': 13, '서': 14, '도': 15, '지': 16, '네': 17, '에': 18, '데': 19, '좋': 20, '구': 21, '사': 22, '기': 23, '해': 24, '은': 25, '로': 26, '만': 27, '나': 28, '무': 29, '보': 30, '있': 31, '매': 32, '게': 33, '~': 34, '리': 35, '그': 36, '잘': 37, '안': 38, '한': 39, '주': 40, '라': 41, '배': 42, '거': 43, '너': 44, '했': 45, '습': 46, '시': 47, '송': 48, '용': 49, '품': 50, '으': 51, '제': 52, '!': 53, '같': 54, '을': 55, '입': 56, '상': 57, 'ㅠ': 58, '들': 59, '인': 60, '자': 61, '면': 62, '재': 63, '스': 64, '정': 65, '먹': 66, '비': 67, '합': 68, '부': 69, '대': 70, '없': 71, '여': 72, '려': 73, '마': 74, '음': 75, '맛': 76, '^': 77, '되': 78, '전': 79, '더': 80, '장': 81, '쓰': 82, '문': 83, '일': 84, '것': 85, 'ㅎ': 86, '건': 87, '저': 88, '않': 89, '수': 90, '생': 91, '르': 92, '빠': 93, '세': 94, '분': 95, '편': 96, '진': 97, '조': 98, '오': 99, '감': 100, '았': 101, '물': 102, '번': 103, '드': 104, '었': 105, '할': 106, '개': 107, '많': 108, '각': 109, '냥': 110, '겠'

In [21]:
# 모든 데이터 수 똑같이 맞춰주기
raw['length'] = raw['review'].str.len()

print(raw.head())
print(raw.describe())


raw['length'][raw['length'] < 100].count()


   rating                                             review  label  length
0       5                                            배공빠르고 굿      1       7
1       2                      택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고      0      29
2       5  아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...      1      68
3       2  선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...      0     136
4       5                  민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ      1      33
              rating          label         length
count  199908.000000  199908.000000  199908.000000
mean        3.225909       0.499995      39.699402
std         1.645431       0.500001      29.168119
min         1.000000       0.000000       3.000000
25%         2.000000       0.000000      17.000000
50%         2.000000       0.000000      29.000000
75%         5.000000       1.000000      55.000000
max         5.000000       1.000000     140.000000


189043

In [24]:
# 모든 문장길이 100자로 제한

from tensorflow.keras.preprocessing.sequence import pad_sequences

X = pad_sequences(train_seq, maxlen=100)

from sklearn.model_selection import train_test_split
trainX, valX, trainY, valY = train_test_split( X, Y, test_size=0.2, random_state=42 )

print(len(trainX))
print(len(valX))

ValueError: Found input variables with inconsistent numbers of samples: [10, 199908]